# Préparation des données pour Projet Final JO - Inégalités H/F

Pour ce projet, je m'occupe de la partie sur l'étude des inégalités Hommes/Femmes aux Jeux Olympiques.  
J'utilise les tables suivantes :
- Les tables "athletes_events.csv" et "noc_regions.csv" du jeu de données "120 years of Olympic history", disponible sur Kaggle :  
    https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results?resource=download
- La table "gender-inequality-index-from-the-human-development-report.csv", disponible sur Our World in Data :  
    https://ourworldindata.org/grapher/gender-inequality-index-from-the-human-development-report

### Importation et nettoyage des fichiers

1. Fichier Athlètes JO

In [1]:
# Importation de la bibliothèque Pandas
import pandas as pd

# Importer les données dans un dataframe
olympic_games = pd.read_csv("./athlete_events.csv", sep = ",")

# Ne garder que les jeux d'été
olympic_games = olympic_games[olympic_games['Season'] == 'Summer']

# Ne garder que les colonnes utiles pour l'étude
olympic_games = olympic_games[['Sex', 'Team', 'NOC', 'Year', 'Sport', 'Medal']]

# Création d'ID (concaténation du NOC et année) pour lier à la table des GII
olympic_games['ID'] = olympic_games['NOC'] + olympic_games['Year'].astype(str)

olympic_games.head()

,Sex,Team,NOC,Year,Sport,Medal,ID
0,M,China,CHN,1992,Basketball,NaN,CHN1992
1,M,China,CHN,2012,Judo,NaN,CHN2012
2,M,Denmark,DEN,1920,Football,NaN,DEN1920
3,M,Denmark/Sweden,DEN,1900,Tug-Of-War,Gold,DEN1900
26,F,Netherlands,NED,1932,Athletics,NaN,NED1932


2. Fichier NOC/Régions

In [2]:
# Importer les données dans un dataframe
noc_regions = pd.read_csv("./noc_regions.csv", sep = ",")

# Trouver les valeurs manquantes dans les colonnes NOC et Region
missing_noc_region = noc_regions[noc_regions[['NOC', 'Region']].isnull().any(axis=1)]
print(missing_noc_region)

# Remplacer les valeurs manquantes dans Region par les valeurs de Notes
noc_regions['Region'].fillna(noc_regions['Notes'], inplace=True)

# Supprimer la colonne Notes
noc_regions = noc_regions.drop('Notes', axis=1)

     NOC Region                 Notes
168  ROT    NaN  Refugee Olympic Team
208  TUV    NaN                Tuvalu
213  UNK    NaN               Unknown


In [3]:
# En visualisation rapide sur Tableau (Map), on voit que la Bolivie n'apparait pas
bolivia_region = noc_regions[noc_regions['NOC'] == 'BOL']
if bolivia_region.empty:
    print("No region information available for NOC code 'BOL'")
else:
    print(bolivia_region)

# Il y a une erreur sur le nom "Boliva" au lieu de "Bolivia"
if not bolivia_region.empty:
    noc_regions.loc[noc_regions['NOC'] == 'BOL', 'Region'] = 'Bolivia'

    NOC  Region
27  BOL  Boliva


In [4]:
# On voit sur Data Wrangler qu'il y a plus de codes NOC que de regions
multiple_noc_regions = noc_regions.groupby('Region')['NOC'].unique().reset_index()
multiple_noc_regions = multiple_noc_regions[multiple_noc_regions['NOC'].str.len() > 1]

print(multiple_noc_regions)

             Region                   NOC
10        Australia            [ANZ, AUS]
33           Canada            [CAN, NFL]
39            China            [CHN, HKG]
48   Czech Republic       [BOH, CZE, TCH]
67          Germany  [FRG, GDR, GER, SAA]
69           Greece            [CRT, GRE]
110        Malaysia       [MAL, MAS, NBO]
152          Russia       [EUN, RUS, URS]
162          Serbia       [SCG, SRB, YUG]
180           Syria            [SYR, UAR]
188        Trinidad            [TTO, WIF]
203         Vietnam            [VIE, VNM]
206           Yemen       [YAR, YEM, YMD]
208        Zimbabwe            [RHO, ZIM]


In [5]:
# On enregistre le dataframe nettoyé à utiliser dans Tableau
noc_regions.to_csv('noc_regions_clean.csv', index=False)

noc_regions.head()

,NOC,Region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


3. Fichier GII index

In [6]:
# Importer les données dans un dataframe
gii_country = pd.read_csv("./gender-inequality-index-from-the-human-development-report.csv", sep = ",")

# Enlever les lignes sans Code
gii_country.dropna(subset=['Code'], inplace=True)

# Renommer la colonne des index
gii_country.rename(columns={'Gender Inequality Index': 'GII'}, inplace=True)

# Création de l'ID (concaténation du NOC et année)
gii_country['ID'] = gii_country['Code'] + gii_country['Year'].astype(str)

gii_country.head()

,Entity,Code,Year,GII,ID
0,Afghanistan,AFG,2005,0.748,AFG2005
1,Afghanistan,AFG,2006,0.749,AFG2006
2,Afghanistan,AFG,2007,0.752,AFG2007
3,Afghanistan,AFG,2008,0.755,AFG2008
4,Afghanistan,AFG,2009,0.755,AFG2009


### Préparation des données

#### 1. Table des médailles et % H/F par sport, NOC et année

Ajout d'une colonne "Type" pour différentier les sports collectifs/individuels

In [7]:
# Faire la liste des sports collectifs
team_sports_list = ['Basketball', 'Football', 'Tug-Of-War', 'Handball', 'Water Polo', 'Hockey', 'Volleyball', 'Synchronized Swimming', 'Baseball', 'Rhythmic Gymnastics', 'Rugby Sevens', 'Beach Volleyball', 'Rugby', 'Lacrosse', 'Polo', 'Cricket', 'Ice Hockey']

# Créer une colonne "Type" avec valeurs par défaut "Individual" 
olympic_games['Type'] = 'Individual'

# Change le type à "Team" pour les sports inclus dans l'array
olympic_games.loc[olympic_games['Sport'].isin(team_sports_list), 'Type'] = 'Team'

olympic_games.head()

,Sex,Team,NOC,Year,Sport,Medal,ID,Type
0,M,China,CHN,1992,Basketball,NaN,CHN1992,Team
1,M,China,CHN,2012,Judo,NaN,CHN2012,Individual
2,M,Denmark,DEN,1920,Football,NaN,DEN1920,Team
3,M,Denmark/Sweden,DEN,1900,Tug-Of-War,Gold,DEN1900,Team
26,F,Netherlands,NED,1932,Athletics,NaN,NED1932,Individual


Créer une nouvelle table groupant par ID, NOC, Year, Sport, Type et afficher les médailles et le % de femmes

In [8]:
# Group by ID, NOC, Year, Sport, and Type and count the total number of athletes
total_athletes_per_group = olympic_games.groupby(['ID', 'NOC', 'Year', 'Sport', 'Type']).size().reset_index(name='Total_Athletes')

# Group by ID, NOC, Year, Sport, and Type and count the number of female and male athletes
female_athletes_per_group = olympic_games[olympic_games['Sex'] == 'F'].groupby(['ID', 'NOC', 'Year', 'Sport', 'Type']).size().reset_index(name='Female_Athletes')
male_athletes_per_group = olympic_games[olympic_games['Sex'] == 'M'].groupby(['ID', 'NOC', 'Year', 'Sport', 'Type']).size().reset_index(name='Male_Athletes')

# Group by ID, NOC, Year, Sport, and Type and count the number of medals
medals_per_group = olympic_games.groupby(['ID', 'NOC', 'Year', 'Sport', 'Type', 'Medal']).size().unstack(fill_value=0).reset_index()

# Calculate the total number of medals per ID, NOC, Year, Sport, and Type
medals_per_group['Total_Medals'] = medals_per_group['Bronze'] + medals_per_group['Gold'] + medals_per_group['Silver']

# Calculate the number of female medals per ID, NOC, Year, Sport, and Type
medals_per_group['Female_Medals'] = medals_per_group['Gold'].where(olympic_games['Sex'] == 'F', 0)

# Merge the four dataframes
merged_df = total_athletes_per_group.merge(female_athletes_per_group, on=['ID', 'NOC', 'Year', 'Sport', 'Type'], how='left')
merged_df = merged_df.merge(male_athletes_per_group, on=['ID', 'NOC', 'Year', 'Sport', 'Type'], how='left')
merged_df = merged_df.merge(medals_per_group[['ID', 'NOC', 'Year', 'Sport', 'Type', 'Total_Medals', 'Female_Medals', 'Gold', 'Silver', 'Bronze']], on=['ID', 'NOC', 'Year', 'Sport', 'Type'], how='left')

# Replace NaN values in Female_Athletes, Male_Athletes, Total_Medals, Female_Medals, Gold, Silver, and Bronze with 0
merged_df['Female_Athletes'].fillna(0, inplace=True)
merged_df['Male_Athletes'].fillna(0, inplace=True)
merged_df['Total_Medals'].fillna(0, inplace=True)
merged_df['Female_Medals'].fillna(0, inplace=True)
merged_df['Gold'].fillna(0, inplace=True)
merged_df['Silver'].fillna(0, inplace=True)
merged_df['Bronze'].fillna(0, inplace=True)

# Convert Female_Athletes, Male_Athletes, Total_Medals, Female_Medals, Gold, Silver, and Bronze columns to integers
merged_df['Female_Athletes'] = merged_df['Female_Athletes'].astype(int)
merged_df['Male_Athletes'] = merged_df['Male_Athletes'].astype(int)
merged_df['Total_Medals'] = merged_df['Total_Medals'].astype(int)
merged_df['Female_Medals'] = merged_df['Female_Medals'].astype(int)
merged_df['Gold'] = merged_df['Gold'].astype(int)
merged_df['Silver'] = merged_df['Silver'].astype(int)
merged_df['Bronze'] = merged_df['Bronze'].astype(int)

# Calculate the percentage of female athletes and the percentage of female medals
merged_df['Percentage_Female'] = (merged_df['Female_Athletes'] / merged_df['Total_Athletes']) * 100
merged_df['Percentage_Female_Medals'] = (merged_df['Female_Medals'] / merged_df['Total_Medals']) * 100

# Display the resulting dataframe
merged_df.head()

,ID,NOC,Year,Sport,Type,Total_Athletes,Female_Athletes,Male_Athletes,Total_Medals,Female_Medals,Gold,Silver,Bronze,Percentage_Female,Percentage_Female_Medals
0,AFG1936,AFG,1936,Athletics,Individual,3,0,3,0,0,0,0,0,0.0,NaN
1,AFG1936,AFG,1936,Hockey,Team,13,0,13,0,0,0,0,0,0.0,NaN
2,AFG1948,AFG,1948,Football,Team,11,0,11,0,0,0,0,0,0.0,NaN
3,AFG1948,AFG,1948,Hockey,Team,14,0,14,0,0,0,0,0,0.0,NaN
4,AFG1956,AFG,1956,Hockey,Team,12,0,12,0,0,0,0,0,0.0,NaN


In [9]:
# Enregistrer le nouveau dataframe dans un fichier csv
merged_df.to_csv('performances_gender_sport.csv', index=False)

#### 2. Table des GII et % femmes par ID

Ajout d'une colonne % d'athlètes femmes

In [10]:
# Group by ID and count occurrences of each ID
grouped_data = olympic_games.groupby('ID').size().reset_index(name='Total')

# Calculate the count of women for each ID
women_count = olympic_games[olympic_games['Sex'] == 'F'].groupby('ID').size().reset_index(name='Female_Count')

# Calculate the percentage of women for each ID
grouped_data = grouped_data.merge(women_count, on='ID', how='left')
grouped_data['Percentage_Female'] = (grouped_data['Female_Count'] / grouped_data['Total']) * 100

# Merge with gii_country dataframe based on ID
gii_country = gii_country.merge(grouped_data[['ID', 'Percentage_Female']], on='ID', how='left')

gii_country.head()

,Entity,Code,Year,GII,ID,Percentage_Female
0,Afghanistan,AFG,2005,0.748,AFG2005,NaN
1,Afghanistan,AFG,2006,0.749,AFG2006,NaN
2,Afghanistan,AFG,2007,0.752,AFG2007,NaN
3,Afghanistan,AFG,2008,0.755,AFG2008,25.0
4,Afghanistan,AFG,2009,0.755,AFG2009,NaN


Ajout d'une colonne nombre de médailles dans la table GII

In [11]:
# Calculate the total number of medals per ID
total_medals = olympic_games.groupby('ID')['Medal'].count().reset_index(name='Total_Medals')

# Calculate the number of medals earned by female athletes per ID
women_medals = olympic_games[olympic_games['Sex'] == 'F'].groupby('ID')['Medal'].count().reset_index(name='Female_Medals')

# Merge all the calculated data with gii_country dataframe based on ID
gii_country = gii_country.merge(total_medals, on='ID', how='left')
gii_country = gii_country.merge(women_medals, on='ID', how='left')

# Remove rows with missing data
gii_country.dropna(inplace=True)

# Convert medals count to integers
gii_country['Total_Medals'] = gii_country['Total_Medals'].astype(int)
gii_country['Female_Medals'] = gii_country['Female_Medals'].astype(int)

gii_country.to_csv('gii_country.csv', index=False)
gii_country.head()

,Entity,Code,Year,GII,ID,Percentage_Female,Total_Medals,Female_Medals
3,Afghanistan,AFG,2008,0.755,AFG2008,25.000000,1,0
7,Afghanistan,AFG,2012,0.738,AFG2012,16.666667,1,0
11,Afghanistan,AFG,2016,0.692,AFG2016,33.333333,0,0
18,Albania,ALB,2000,0.319,ALB2000,60.000000,0,0
22,Albania,ALB,2004,0.301,ALB2004,28.571429,0,0
